In [9]:
# Include necessary imports
import os
import torch 
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from torch.nn.utils.rnn import pad_sequence
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [10]:
# Preprocess the data

# Transpose all valies to the same key
def key_transposition(df):
    transpose_val = 48 - df.iloc[3][0]
    df += transpose_val
    return df

# Min-Max normalization technique
def normalize_df(df):
    X_std = (df - 1) / (88 - 1)
    return X_std

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                transposed_df = key_transposition(df.transpose())
                normalized_df = normalize_df(transposed_df)
                if dirname == 'test':
                    test.append(normalized_df)
                if dirname == 'train':
                    train.append(normalized_df)
                if dirname == 'valid':
                    validation.append(normalized_df)

# Model

In [4]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=40, n_layers=2, dropout_rate=.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.dropout = torch.nn.Dropout(dropout_rate)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        lstm_output = self.dropout(lstm_output)
        model_output = self.fc(lstm_output)
        model_output = self.sigmoid(model_output)
        return model_output

# Train

In [13]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()
    for song_index, song in enumerate(train[:10]):
        print(f"Training on song {song_index + 1}")
        
        melody = torch.tensor(song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            output = model(melody)
            loss = criterion(output, harmonies)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
                if torch.isnan(loss):
                    print("harmonies: ", harmonies)
                    print("melody:" , melody)
                    print("output: ", output)
                    

        model.eval()
        total_val_loss = 0
        
        # validation songs
        with torch.no_grad():
            for val_song in validation:
                val_melody = torch.tensor(val_song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
                val_harmonies = torch.tensor(val_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
                
                val_output = model(val_melody)
                val_loss = criterion(val_output, val_harmonies)
                total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [ ]:
criterion = torch.nn.MSELoss()
num_epochs = 5000

model = Model(1, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_model(model, optimizer, criterion, num_epochs)

Training on song 1
torch.Size([1, 180, 1])
torch.Size([1, 180, 3])
Song 1, Epoch 100/5000, Loss: 0.0014270960818976164
Song 1, Epoch 200/5000, Loss: 0.0013960956130176783
Song 1, Epoch 300/5000, Loss: 0.0014213745016604662
Song 1, Epoch 400/5000, Loss: 0.0013838415034115314
Song 1, Epoch 500/5000, Loss: 0.0013468981487676501
Song 1, Epoch 600/5000, Loss: 0.001387500436976552
Song 1, Epoch 700/5000, Loss: 0.0013233496574684978
Song 1, Epoch 800/5000, Loss: 0.0013890728587284684
Song 1, Epoch 900/5000, Loss: 0.0013641389086842537
Song 1, Epoch 1000/5000, Loss: 0.0013794498518109322
Song 1, Epoch 1100/5000, Loss: 0.001373840612359345
Song 1, Epoch 1200/5000, Loss: 0.0013554851757362485
Song 1, Epoch 1300/5000, Loss: 0.0013431255938485265
Song 1, Epoch 1400/5000, Loss: 0.0013310563517734408
Song 1, Epoch 1500/5000, Loss: 0.0012865348253399134
Song 1, Epoch 1600/5000, Loss: 0.0013917596079409122
Song 1, Epoch 1700/5000, Loss: 0.001375482534058392
Song 1, Epoch 1800/5000, Loss: 0.00138955994

# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}

for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [78]:
def inverse_df(df):
    X_scaled = df * (88 - 1) + 1
    return X_scaled

def midi_to_note(part):
    result = stream.Part()
    count = 1
    prev = round(part[0])
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

def output_to_sheet_music(melody, result):
    result_numpy = result.detach().numpy()
    melody = inverse_df(melody)
    inversed = inverse_df(np.squeeze(result_numpy)).T
    
    score = stream.Score()
    melody_part = midi_to_note(melody)
    
    alto_notes = inversed[0]
    tenor_notes = inversed[1]
    bass_notes = inversed[2]  
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [107]:
 test_song = test[0]
melody = test_song.iloc[0]
harmonies = torch.tensor(test_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
result = model(torch.tensor(test_song.iloc[0].values.reshape(-1,1), dtype=torch.float32).unsqueeze(0).reshape(1,test_song.shape[1],1))
output_to_sheet_music(melody, result)

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss